首先 import 必要的模块

In [1]:
import lightgbm as lgbm
#from lgbm.sklearn import LGBMClassifier
import xlearn as xl
import pandas as pd 
import numpy as np
import time

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss     #采用logloss作为评价指标
from pandas import DataFrame

from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder
from matplotlib import pyplot
import seaborn as sns
%matplotlib inline
from libffm_data_generate import *

## 1.1读取数据

In [2]:
train_name_1='800_0'
train_name_2='800_1'
valid_name_1='800_2'
valid_name_2='800_3'

In [4]:
#train_data
train_1 = pd.read_csv("/home/jupyter/训练数据组/400w数据组/"+train_name_1,index_col=0).reset_index(drop=True)   #拿200万条数据训练
train_1.drop(['id','hour','user_id','Unnamed: 0'],axis=1,inplace=True)
print(train_1.shape)
#validation_data
valid_1 = pd.read_csv("/home/jupyter/训练数据组/400w数据组/"+valid_name_1,index_col=0).reset_index(drop=True)   #拿200万条数据训练
valid_1.drop(['id','hour','user_id','Unnamed: 0'],axis=1,inplace=True)
print(valid_1.shape)

(3996246, 23)
(3886316, 23)


In [30]:
#train_data
train_2 = pd.read_csv("/home/jupyter/训练数据组/100w数据组/"+train_name_2,index_col=0).reset_index(drop=True)   #拿200万条数据训练
train_2.drop(['id','hour','user_id','Unnamed: 0'],axis=1,inplace=True)
print(train_2.shape)
valid_2 = pd.read_csv("/home/jupyter/训练数据组/100w数据组/"+valid_name_2,index_col=0).reset_index(drop=True)   #拿200万条数据训练
valid_2.drop(['id','hour','user_id','Unnamed: 0'],axis=1,inplace=True)
print(valid_2.shape)

(999864, 23)
(996224, 23)


In [7]:
#test_data
test = pd.read_csv("/home/jupyter/训练数据组/FE_test_26_final.csv",index_col=0).reset_index(drop=True)   #拿200万条数据训练
test.drop(['id','hour','user_id'],axis=1,inplace=True)
test.shape

/home/qiaoliang/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(4218938, 23)

In [4]:
#first
y_train_1 = train_1['click']
x_train_1 = train_1.drop(["click"], axis=1)

y_valid_1 = valid_1['click']
x_valid_1 = valid_1.drop(["click"], axis=1)

In [6]:
#second
y_train_2 = train_2['click']
x_train_2 = train_2.drop(["click"], axis=1)

y_valid_2 = valid_2['click']
x_valid_2 = valid_2.drop(["click"], axis=1)

In [8]:
y_test = test['click']
x_test = test.drop(["click"], axis=1)

In [14]:
y_test.shape

(4218938,)

### 1.2 original_data generate libffm

In [6]:
#ffm-data
#train
start=time.time()
original_ffm_data(x=x_train_1,y=y_train_1,save_name=r'/home/jupyter/xlearn/ffm/ffm_'+train_name_1+'.txt')
end=time.time()
print(end-start)
#valid
start=time.time()
original_ffm_data(x=x_valid_1,y=y_valid_1,save_name=r'/home/jupyter/xlearn/ffm/ffm_'+valid_name_1+'.txt')
end=time.time()
print(end-start)

157.61609506607056
157.64250016212463


In [16]:
aa=pd.read_csv(r'/home/jupyter/xlearn/ffm/'+'ffm_test.txt')

In [18]:
aa.shape

(4218937, 1)

In [41]:
#test
start=time.time()
original_ffm_data(x=x_test,y=y_test,save_name=r'/home/jupyter/xlearn/ffm/'+'ffm_test.txt')
end=time.time()
print(end-start)

174.09121370315552
173.4769024848938
754.1248562335968


In [32]:
#fm_data
x_train_fm_1=np.array(x_train_1)
x_valid_fm_1=np.array(x_valid_1)
x_test_fm_1=np.array(x_test)
xl.write_data_to_xlearn_format(X=x_train_fm_1,y=y_train_1,filepath=r'/home/jupyter/xlearn/fm'+train_1_name+'.txt')
xl.write_data_to_xlearn_format(X=x_valid_fm_1,y=y_valid_1,filepath=r'/home/jupyter/xlearn/fm'+valid_name_1+'.txt')
xl.write_data_to_xlearn_format(X=x_test_fm_1,y=y_test,filepath=r'/home/jupyter/xlearn/fm'+'fm_test.txt')

In [30]:
del train_1
del train_2
del valid_1
del valid_2
del test

默认参数，此时学习率为0.1，比较大，观察弱分类数目的大致范围
（采用默认参数配置，看看模型是过拟合还是欠拟合）

### 2. lightgbm train

In [7]:
start=time.time()
# 定义模型
lgb1 = lgbm.LGBMRegressor(
          num_boost_round=50,
          boosting='gbdt', 
          objective='binary', 
          num_threads=6, 
          #default=OpenMP_default, type=int, alias=num_thread, nthread LightGBM 的线程数
#          'silent': True,
          learning_rate=0.1, 
          num_leaves=24, 
          #num_leaves：叶子数
#num_leaves, default=31, type=int, alias=num_leaf一棵树上的叶子数
          max_depth= 5,
          max_bi= 256, 
          subsample_for_bin= 200000,#默认200000，用来构建直方图数据数量
          subsample= 0.8, #默认1
          #类似于feature_fraction,它将在不进行重采样的情况下
          #随机选择部分数据可以用来加速训练可以用来处理过拟合Not
          subsample_freq= 1, 
          colsample_bytree= 0.8,       
          reg_alpha= 0,         
          reg_lambda= 0, 
          min_split_gain=0.0, 
          min_child_weight= 1,        
          min_child_samples= 100, 
          min_data_in_leaf=50, 
          #default=20, type=int, alias=min_data_per_leaf , min_data, 
          #min_child_samples一个叶子上数据的最小数量. 可以用来处理过拟合.
        
        # 'categorical_feature':3
         # 'categorical_feature':3:26
         )
# 训练学习
model=lgb1.fit(x_train_1, y_train_1)
end=time.time()
print(end-start)

/home/qiaoliang/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


2.389890193939209


In [8]:
# train上logloss评测
y_pred_train = lgb1.predict(x_train_1)
logloss = log_loss(y_train_1, y_pred_train)
print ("logloss of train :",logloss )

# valid上logloss评测
y_pred_valid = lgb1.predict(x_valid_1)
logloss = log_loss(y_valid_1, y_pred_valid)
print ("logloss of valid :",logloss )

# test上logloss评测
y_pred_test = lgb1.predict(x_test)
logloss = log_loss(y_test, y_pred_test)
print ("logloss of test :",logloss )

logloss of train : 0.41508363815322297
logloss of valid : 0.4154014815745415


NameError: name 'x_test' is not defined

In [64]:
x_train_leaves = lgb1.predict(x_train_1, pred_leaf=True)    
x_valid_leaves = lgb1.predict(x_valid_1, pred_leaf=True) 
x_test_leaves = lgb1.predict(x_test, pred_leaf=True) 
print(x_train_leaves.shape)
print(x_valid_leaves.shape)
print(x_test_leaves.shape)

(999864, 50)
(999864, 50)
(4218938, 50)


In [9]:
x_train_leaves = lgb1.predict(x_train_2, pred_leaf=True)    
x_valid_leaves = lgb1.predict(x_valid_2, pred_leaf=True) 
print(x_train_leaves.shape)
print(x_valid_leaves.shape)

(999864, 50)
(996224, 50)


### 1、生成libffm所需的格式

### $$$ 执行代码

In [ ]:
start=time.time()
ffm_data_t(x_train_leaves=x_train_leaves,y_train=y_train,save_name=r'/usr/local/ffm_train.txt')
end=time.time()
print(end-start)

start=time.time()
ffm_data_t(x_train_leaves=x_test_leaves,y_train=y_test,save_name=r'/usr/local/ffm_test.txt')
end=time.time()
print(end-start)

427.5567891597748


## x-learn train

### 1. from leaf to libffm data

In [40]:
y_train_1.shape

(999864,)

In [10]:
#用于gbm训练的数据，直接进入ffm
# self def to produce libffm data
start=time.time()
xl.write_data_to_xlearn_format(X=x_train_leaves,y=y_train_2,filepath=r'/home/jupyter/xlearn/fm_train.txt')
xl.write_data_to_xlearn_format(X=x_valid_leaves,y=y_valid_2,filepath=r'/home/jupyter/xlearn/fm_valid.txt')

In [ ]:
xl.write_data_to_xlearn_format(X=x_test_leaves,y=y_test,filepath=r'/home/jupyter/xlearn/fm_test.txt')
end=time.time()
print(end-start)

### 2.ffm模型训练-100w数据集

In [17]:
# 原始数据进入ffm初始化
ffm_model = xl.create_ffm() 
ffm_model.setTrain(r'/home/jupyter/xlearn/ffm/ffm_'+train_name_1+'.txt')  # 设置训练数据
ffm_model.setValidate(r'/home/jupyter/xlearn/ffm/ffm_'+valid_name_1+'.txt')  #设置验证数据
ffm_model.setTest(r'/home/jupyter/xlearn/ffm/ffm_test.txt')  # test data

In [11]:
# lightgbm 进入ffm初始化
#ffm_model = xl.create_ffm() # Use field-aware factorization machine
#ffm_model.setTrain(r'/home/jupyter/xlearn/fm_train.txt')  # 设置训练数据
#ffm_model.setValidate(r'/home/jupyter/xlearn/fm_valid.txt')  #设置验证数据
#ffm_model.setTest(r'/home/jupyter/xlearn/fm_test.txt') 

In [26]:
# param:
#  0. binary classification
#  1. learning rate: 0.2
#  2. regular lambda: 0.002
#  3. evaluation metric: accuracystart=time.time()
param = {'task':'binary','k':4,'opt':'adagrad','lr':0.2,'lambda':0.0001, 
         'metric':'acc','fold':2,'epoch':26,'stop_window':3}

#ffm_model.disableNorm()    # disable Instance-wise Normalization

ffm_model.setSign()  # Convert output to 0-1  scope
result=ffm_model.fit(param, r'/home/jupyter/xlearn/ffm/model.out')
result_cv=ffm_model.cv(param)
# Prediction task
end=time.time()
print(end-start)

1236.9563615322113


In [27]:
time.sleep(5)
ffm_model.predict(r'/home/jupyter/xlearn/ffm/model.out', r'/home/jupyter/xlearn/output.txt')
#0.40547

### 3、opt选择连接ftrl调优-100w数据集

In [28]:
param = {'task':'binary','k':4,'opt':'ftrl','lr':0.2,'lambda':0.0001, 
         'metric':'acc','fold':2,'epoch':26,'stop_window':3,
        'alpha':0.002,'beta':0.8,'lambda_1':0.001,'lambda_2':1}

ffm_model.setSign()  # Convert output to 0-1  scope
result=ffm_model.fit(param, r'/home/jupyter/xlearn/ffm/model.out')
result_cv=ffm_model.cv(param)
# Prediction task
end=time.time()
print(end-start)

1780.3661370277405


In [29]:
param = {'task':'binary','k':4,'opt':'ftrl','lr':0.2,'lambda':0.0001, 
         'metric':'acc','fold':2,'epoch':26,'stop_window':3,
        'alpha':0.002,'beta':0.8,'lambda_1':0.001,'lambda_2':0.1}

ffm_model.setSign()  # Convert output to 0-1  scope
result=ffm_model.fit(param, r'/home/jupyter/xlearn/ffm/model.out')
result_cv=ffm_model.cv(param)
# Prediction task
end=time.time()
print(end-start)

2026.9213027954102


### 4、ffm训练在400w数据集上

##### 4.1

In [ ]:
# 原始数据进入ffm初始化
ffm_model = xl.create_ffm() 
ffm_model.setTrain(r'/home/jupyter/xlearn/ffm/ffm_'+train_name_1+'.txt')  # 设置训练数据
ffm_model.setValidate(r'/home/jupyter/xlearn/ffm/ffm_'+valid_name_1+'.txt')  #设置验证数据
ffm_model.setTest(r'/home/jupyter/xlearn/ffm/ffm_test.txt')  # test data

In [25]:
start=time.time()
# param:
#  0. binary classification
#  1. learning rate: 0.2
#  2. regular lambda: 0.002
#  3. evaluation metric: accuracystart=time.time()
param = {'task':'binary','k':4,'opt':'adagrad','lr':0.2,'lambda':0.0001, 
         'metric':'acc','fold':2,'epoch':26,'stop_window':3}

#ffm_model.disableNorm()    # disable Instance-wise Normalization
ffm_model.setSigmoid()
#ffm_model.setSign()  # Convert output to 0-1  scope
result=ffm_model.fit(param, r'/home/jupyter/xlearn/ffm/model_400.out')
result_cv=ffm_model.cv(param)
# Prediction task
end=time.time()
print(end-start)

285.26229786872864


In [27]:
time.sleep(5)
ffm_model.setSigmoid()
ffm_model.predict(r'/home/jupyter/xlearn/ffm/model_400.out', r'/home/jupyter/xlearn/ffm/output_400.txt')
#0.40547
y_ffm_predict=pd.read_csv(r'/home/jupyter/xlearn/ffm/output_400.txt',header=None)
logloss = log_loss(y_test, y_ffm_predict)
print ("logloss of test :",logloss )

logloss of test : 0.40499725689989713


#### 4_2_1 lambda=0.00001

In [17]:
start=time.time()
# param:
#  0. binary classification
#  1. learning rate: 0.2
#  2. regular lambda: 0.002
#  3. evaluation metric: accuracystart=time.time()
param = {'task':'binary','k':4,'opt':'adagrad','lr':0.2,'lambda':0.00001, 
         'metric':'acc','fold':2,'epoch':26,'stop_window':3}

#ffm_model.disableNorm()    # disable Instance-wise Normalization
ffm_model.setSigmoid()
#ffm_model.setSign()  # Convert output to 0-1  scope
result=ffm_model.fit(param, r'/home/jupyter/xlearn/ffm/model_400.out')
result_cv=ffm_model.cv(param)
# Prediction task
end=time.time()
print(end-start)


260.5236761569977


In [16]:
time.sleep(5)
ffm_model.setSigmoid()
ffm_model.predict(r'/home/jupyter/xlearn/ffm/model_400.out', r'/home/jupyter/xlearn/ffm/output_400.txt')
#0.40547
y_ffm_predict=pd.read_csv(r'/home/jupyter/xlearn/ffm/output_400.txt',header=None)
logloss = log_loss(y_test, y_ffm_predict)
print ("logloss of test :",logloss )

logloss of test : 0.40565516652332156


#### lambda=0.000025

In [73]:
start=time.time()
# param:
#  0. binary classification
#  1. learning rate: 0.2
#  2. regular lambda: 0.002
#  3. evaluation metric: accuracystart=time.time()
param = {'task':'binary','k':4,'opt':'adagrad','lr':0.2,'lambda':0.000025, 
         'metric':'acc','fold':2,'epoch':26,'stop_window':3}

#ffm_model.disableNorm()    # disable Instance-wise Normalization
ffm_model.setSigmoid()
#ffm_model.setSign()  # Convert output to 0-1  scope
result=ffm_model.fit(param, r'/home/jupyter/xlearn/ffm/model_400.out')
result_cv=ffm_model.cv(param)
# Prediction task
end=time.time()
print(end-start)

time.sleep(15)
ffm_model.setSigmoid()
ffm_model.predict(r'/home/jupyter/xlearn/ffm/model_400.out', r'/home/jupyter/xlearn/ffm/output_400.txt')
#0.40547
y_ffm_predict=pd.read_csv(r'/home/jupyter/xlearn/ffm/output_400.txt',header=None)
logloss = log_loss(y_test, y_ffm_predict)
print ("logloss of test :",logloss )

296.2349212169647
logloss of test : 0.40428048256343824


#### lambda=0.00005

In [7]:
start=time.time()
# param:
#  0. binary classification
#  1. learning rate: 0.2
#  2. regular lambda: 0.002
#  3. evaluation metric: accuracystart=time.time()
param = {'task':'binary','k':4,'opt':'adagrad','lr':0.2,'lambda':0.00005, 
         'metric':'acc','fold':2,'epoch':26,'stop_window':3}

#ffm_model.disableNorm()    # disable Instance-wise Normalization
ffm_model.setSigmoid()
#ffm_model.setSign()  # Convert output to 0-1  scope
result=ffm_model.fit(param, r'/home/jupyter/xlearn/ffm/model_400.out')
result_cv=ffm_model.cv(param)
# Prediction task
end=time.time()
print(end-start)

319.87837266921997


In [8]:
time.sleep(15)
#ffm_model.setSigmoid()
ffm_model.predict(r'/home/jupyter/xlearn/ffm/model_400.out', r'/home/jupyter/xlearn/ffm/output_400.txt')
#0.40547
y_ffm_predict=pd.read_csv(r'/home/jupyter/xlearn/ffm/output_400.txt',header=None)
logloss = log_loss(y_test, y_ffm_predict)
print ("logloss of test :",logloss )

logloss of test : 0.40474737996021176


#### lambda=0.0002

In [9]:
start=time.time()
# param:
#  0. binary classification
#  1. learning rate: 0.2
#  2. regular lambda: 0.002
#  3. evaluation metric: accuracystart=time.time()
param = {'task':'binary','k':4,'opt':'adagrad','lr':0.2,'lambda':0.0002, 
         'metric':'acc','fold':2,'epoch':26,'stop_window':3}

#ffm_model.disableNorm()    # disable Instance-wise Normalization
ffm_model.setSigmoid()
#ffm_model.setSign()  # Convert output to 0-1  scope
result=ffm_model.fit(param, r'/home/jupyter/xlearn/ffm/model_400.out')
result_cv=ffm_model.cv(param)
# Prediction task
end=time.time()
print(end-start)

time.sleep(15)
#ffm_model.setSigmoid()
ffm_model.predict(r'/home/jupyter/xlearn/ffm/model_400.out', r'/home/jupyter/xlearn/ffm/output_400.txt')
#0.40547
y_ffm_predict=pd.read_csv(r'/home/jupyter/xlearn/ffm/output_400.txt',header=None)
logloss = log_loss(y_test, y_ffm_predict)
print ("logloss of test :",logloss )

285.56197237968445
logloss of test : 0.4051638143060699


#### epoch

In [25]:
start=time.time()
# param:
#  0. binary classification
#  1. learning rate: 0.2
#  2. regular lambda: 0.002
#  3. evaluation metric: accuracystart=time.time()
param = {'task':'binary','k':4,'opt':'adagrad','lr':0.2,'lambda':0.0001, 
         'metric':'acc','fold':2,'epoch':20,'stop_window':3}

#ffm_model.disableNorm()    # disable Instance-wise Normalization
ffm_model.setSigmoid()
#ffm_model.setSign()  # Convert output to 0-1  scope
result=ffm_model.fit(param, r'/home/jupyter/xlearn/ffm/model_400.out')
result_cv=ffm_model.cv(param)
# Prediction task
end=time.time()
print(end-start)

time.sleep(15)


238.81746816635132


In [23]:
#ffm_model.setSigmoid()
ffm_model.predict(r'/home/jupyter/xlearn/ffm/model_400.out', r'/home/jupyter/xlearn/ffm/output_400.txt')
#0.40547
y_ffm_predict=pd.read_csv(r'/home/jupyter/xlearn/ffm/output_400.txt',header=None)
logloss = log_loss(y_test, y_ffm_predict)
print ("logloss of test :",logloss )

logloss of test : 0.4047878934880913


#### ffm在800万数据集上

In [3]:
# 原始数据进入ffm初始化
ffm_model = xl.create_ffm() 
ffm_model.setTrain(r'/home/jupyter/xlearn/ffm/ffm_'+train_name_1+'.txt')  # 设置训练数据
ffm_model.setValidate(r'/home/jupyter/xlearn/ffm/ffm_'+valid_name_1+'.txt')  #设置验证数据
ffm_model.setTest(r'/home/jupyter/xlearn/ffm/ffm_test.txt')  # test data

In [4]:
start=time.time()
# param:
#  0. binary classification
#  1. learning rate: 0.2
#  2. regular lambda: 0.002
#  3. evaluation metric: accuracystart=time.time()
param = {'task':'binary','k':4,'opt':'adagrad','lr':0.2,'lambda':0.0001, 
         'metric':'acc','fold':2,'epoch':26,'stop_window':3}

#ffm_model.disableNorm()    # disable Instance-wise Normalization
ffm_model.setSigmoid()
#ffm_model.setSign()  # Convert output to 0-1  scope
result=ffm_model.fit(param, r'/home/jupyter/xlearn/ffm/model_400.out')
result_cv=ffm_model.cv(param)
# Prediction task
end=time.time()
print(end-start)

670.4300518035889


In [10]:
time.sleep(5)
#ffm_model.setSigmoid()
ffm_model.predict(r'/home/jupyter/xlearn/ffm/model_400.out', r'/home/jupyter/xlearn/ffm/output_400.txt')
#0.40547
y_ffm_predict=pd.read_csv(r'/home/jupyter/xlearn/ffm/output_400.txt',header=None)
logloss = log_loss(y_test, y_ffm_predict)
print ("logloss of test :",logloss )
#

logloss of test : 0.4060380666747183


#### 'lambda':0.00005

In [11]:
start=time.time()
# param:
#  0. binary classification
#  1. learning rate: 0.2
#  2. regular lambda: 0.002
#  3. evaluation metric: accuracystart=time.time()
param = {'task':'binary','k':4,'opt':'adagrad','lr':0.2,'lambda':0.00005, 
         'metric':'acc','fold':2,'epoch':26,'stop_window':3}

#ffm_model.disableNorm()    # disable Instance-wise Normalization
ffm_model.setSigmoid()
#ffm_model.setSign()  # Convert output to 0-1  scope
result=ffm_model.fit(param, r'/home/jupyter/xlearn/ffm/model_400.out')
result_cv=ffm_model.cv(param)
# Prediction task
end=time.time()
print(end-start)

579.3012561798096


In [13]:
time.sleep(5)
#ffm_model.setSigmoid()
ffm_model.predict(r'/home/jupyter/xlearn/ffm/model_400.out', r'/home/jupyter/xlearn/ffm/output_400.txt')
#0.40547
y_ffm_predict=pd.read_csv(r'/home/jupyter/xlearn/ffm/output_400.txt',header=None)
logloss = log_loss(y_test, y_ffm_predict)
print ("logloss of test :",logloss )
#

logloss of test : 0.4048570572361045


##### 'lambda':0.00001

In [14]:
start=time.time()
# param:
#  0. binary classification
#  1. learning rate: 0.2
#  2. regular lambda: 0.002
#  3. evaluation metric: accuracystart=time.time()
param = {'task':'binary','k':4,'opt':'adagrad','lr':0.2,'lambda':0.00001, 
         'metric':'acc','fold':2,'epoch':26,'stop_window':3}

#ffm_model.disableNorm()    # disable Instance-wise Normalization
ffm_model.setSigmoid()
#ffm_model.setSign()  # Convert output to 0-1  scope
result=ffm_model.fit(param, r'/home/jupyter/xlearn/ffm/model_400.out')
result_cv=ffm_model.cv(param)
# Prediction task
end=time.time()
print(end-start)

573.906849861145


In [15]:
time.sleep(5)
#ffm_model.setSigmoid()
ffm_model.predict(r'/home/jupyter/xlearn/ffm/model_400.out', r'/home/jupyter/xlearn/ffm/output_400.txt')
#0.40547
y_ffm_predict=pd.read_csv(r'/home/jupyter/xlearn/ffm/output_400.txt',header=None)
logloss = log_loss(y_test, y_ffm_predict)
print ("logloss of test :",logloss )

logloss of test : 0.4089581619953243


#### 'lambda':0.000075

In [16]:
start=time.time()
# param:
#  0. binary classification
#  1. learning rate: 0.2
#  2. regular lambda: 0.002
#  3. evaluation metric: accuracystart=time.time()
param = {'task':'binary','k':4,'opt':'adagrad','lr':0.2,'lambda':0.000075, 
         'metric':'acc','fold':2,'epoch':26,'stop_window':3}

#ffm_model.disableNorm()    # disable Instance-wise Normalization
ffm_model.setSigmoid()
#ffm_model.setSign()  # Convert output to 0-1  scope
result=ffm_model.fit(param, r'/home/jupyter/xlearn/ffm/model_400.out')
result_cv=ffm_model.cv(param)
# Prediction task
end=time.time()
print(end-start)

561.7510893344879


In [17]:
time.sleep(5)
#ffm_model.setSigmoid()
ffm_model.predict(r'/home/jupyter/xlearn/ffm/model_400.out', r'/home/jupyter/xlearn/ffm/output_400.txt')
#0.40547
y_ffm_predict=pd.read_csv(r'/home/jupyter/xlearn/ffm/output_400.txt',header=None)
logloss = log_loss(y_test, y_ffm_predict)
print ("logloss of test :",logloss )

logloss of test : 0.4060936461077703


#### epoch

In [18]:
start=time.time()
# param:
#  0. binary classification
#  1. learning rate: 0.2
#  2. regular lambda: 0.002
#  3. evaluation metric: accuracystart=time.time()
param = {'task':'binary','k':4,'opt':'adagrad','lr':0.2,'lambda':0.000075, 
         'metric':'acc','fold':2,'epoch':50,'stop_window':3}

#ffm_model.disableNorm()    # disable Instance-wise Normalization
ffm_model.setSigmoid()
#ffm_model.setSign()  # Convert output to 0-1  scope
result=ffm_model.fit(param, r'/home/jupyter/xlearn/ffm/model_400.out')
result_cv=ffm_model.cv(param)
# Prediction task
end=time.time()
print(end-start)

1241.8230800628662


In [ ]:
time.sleep(5)
#ffm_model.setSigmoid()
ffm_model.predict(r'/home/jupyter/xlearn/ffm/model_400.out', r'/home/jupyter/xlearn/ffm/output_400.txt')
#0.40547
y_ffm_predict=pd.read_csv(r'/home/jupyter/xlearn/ffm/output_400.txt',header=None)
logloss = log_loss(y_test, y_ffm_predict)
print ("logloss of test :",logloss )

In [4]:
for x in range(1,5,2):
    print(x)

1
3
